In [1]:
import cv2
import requests 
import shutil 
import time
import os
import numpy as np

In [2]:
import os
import time 
import cv2
import face_recognition
import numpy as np
from skimage import io
from sklearn import svm
from joblib import dump, load
from PIL import Image, ImageDraw
from sklearn.metrics import classification_report,accuracy_score


In [3]:
import csv

def append_row_to_csv(file_path, row):
    with open(file_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(row)

In [4]:
path='E:/billboard/'

In [5]:
def update_records(ID,GENDER,AGE):
    import datetime

    # Get the current date and time
    current_datetime = datetime.datetime.now()
    current_date = datetime.date.today()

    # Print current date
    print("Current date is:", current_date)
    # Get the first day of the current month
    first_day_of_month = datetime.datetime(current_datetime.year, current_datetime.month, 1)

    # Calculate the number of weeks passed in the current month
    weeks_passed_in_month = (current_datetime.day - 1) // 7 + 1

    # Print the results
    print("Current Day:", current_datetime.strftime("%A"))
    print("Current Time:", current_datetime.strftime("%H:%M:%S"))
    print("Week of the Month:", weeks_passed_in_month)
    day=str(current_datetime.strftime("%A")).upper()
    tym=int(current_datetime.strftime("%H"))
    
    print(tym,type(tym))
    tym1=''+str(tym)+"-"+str(tym+1)
    print(ID,GENDER,AGE,current_date,tym1,day,'Week '+str(weeks_passed_in_month))
    file_path = 'billboard_dataset.csv'
    row_to_append = [ID,GENDER,AGE,current_date,tym1,day,'Week '+str(weeks_passed_in_month)]
    append_row_to_csv(file_path, row_to_append)
    


In [6]:
update_records(10,"Male","Adult")

Current date is: 2024-04-23
Current Day: Tuesday
Current Time: 17:13:28
Week of the Month: 4
17 <class 'int'>
10 Male Adult 2024-04-23 17-18 TUESDAY Week 4


In [7]:
def face_recognition(input_image,count):
    folders = []
    import face_recognition
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path+"dataset"):
        for folder in d:
            folders.append(folder)
    print(folders)
    known_face_encodings=[]
    known_face_names=[]
    for f in folders:
        ifile=path+'Dataset/'+f+'/0.jpg'
        print(ifile)
        ru_image = face_recognition.load_image_file(ifile)
        ru_face_encoding = face_recognition.face_encodings(ru_image)[0]
        known_face_encodings.append(ru_face_encoding)
        known_face_names.append(f)
    print('done extraction')
    clf = svm.SVC(gamma='scale')
    clf.fit(known_face_encodings,known_face_names)
    dump(clf,'SVM.Model')
    print('done training')
    try:
        

        unknown_image = face_recognition.load_image_file(input_image)
        face_locations = face_recognition.face_locations(unknown_image)
        face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

        pil_image = Image.fromarray(unknown_image)
        draw = ImageDraw.Draw(pil_image)
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            print('inside for')
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown Person"

            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))
            text_width, text_height = draw.textsize(name)
            print("The Person is:",name)
        if name == "Unknown Person":
            
            un_image = cv2.imread(input_image)
            cv2.imwrite("Unknown.jpg",un_image)
            #pil_image.save("Unknown.jpg")
            f=open("result.txt","w")
            print(("unknown"),file = f)
            f.close()

            

            folders = []

            # r=root, d=directories, f = files
            for r, d, f in os.walk(path+'Dataset'):
                for folder in d:
                    folders.append(folder)
            print(folders)
            os.mkdir(path+'Dataset/'+str(len(folders))) 
            cv2.imwrite(path+'Dataset/'+str(len(folders))+"/0.jpg",un_image)
            known_face_encodings=[]
            known_face_names=[]
            for f in folders:
                ifile=path+'Dataset/'+str(len(folders))+'/0.jpg'
                print(ifile)
                ru_image = face_recognition.load_image_file(ifile)
                ru_face_encoding = face_recognition.face_encodings(ru_image)[0]
                known_face_encodings.append(ru_face_encoding)
                known_face_names.append(f)
            clf = svm.SVC(gamma='scale')
            print(len(known_face_encodings),len(known_face_names))
            clf.fit(known_face_encodings,known_face_names)
            dump(clf,'SVM.Model')
            print('Retrained the model')
        else:
            f=open("result.txt","w")
            str1=f'face_matched:{name}'
            print(str1,file = f)
            f.close()
        uid=name
        f=open('E:/billboard/output/'+str(count)+'_age.txt','r')
        ag=f.read()
        f.close()
        f=open('E:/billboard/output/'+str(count)+'_gender.txt','r')
        gen=f.read()
        f.close()
        update_records(uid,ag,gen)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))

    except:
        pass

In [8]:
#face_recognition(path+'object_opencv/detected_faces/4.png')

In [9]:
def highlightFace(net, frame, conf_threshold=0.7):
    frameOpencvDnn=frame.copy()
    frameHeight=frameOpencvDnn.shape[0]
    frameWidth=frameOpencvDnn.shape[1]
    blob=cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections=net.forward()
    faceBoxes=[]
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>conf_threshold:
            x1=int(detections[0,0,i,3]*frameWidth)
            y1=int(detections[0,0,i,4]*frameHeight)
            x2=int(detections[0,0,i,5]*frameWidth)
            y2=int(detections[0,0,i,6]*frameHeight)
            faceBoxes.append([x1,y1,x2,y2])
            cv2.rectangle(frameOpencvDnn, (x1,y1), (x2,y2), (0,255,0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn,faceBoxes

In [10]:
faceProto=path+"gender/opencv_face_detector.pbtxt"
faceModel=path+"gender/opencv_face_detector_uint8.pb"
ageProto=path+"gender/age_deploy.prototxt"
ageModel=path+"gender/age_net.caffemodel"
genderProto=path+"gender/gender_deploy.prototxt"
genderModel=path+"gender/gender_net.caffemodel"

MODEL_MEAN_VALUES=(78.4263377603, 87.7689143744, 114.895847746)
ageList=['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList=['Male','Female']

In [11]:
faceNet=cv2.dnn.readNet(faceModel,faceProto)
ageNet=cv2.dnn.readNet(ageModel,ageProto)
genderNet=cv2.dnn.readNet(genderModel,genderProto)

In [12]:
padding=20

In [13]:
output_dir = path+'final/detected_faces'
os.makedirs(output_dir, exist_ok=True)

# Load the pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
def detect_and_save_faces(frame, frame_count):
    try:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
        for i, (x, y, w, h) in enumerate(faces):
            face = frame[y:y+h, x:x+w]
            #face_file = os.path.join(output_dir, f'face_{frame_count}_{i}.jpg')
            cv2.imwrite(output_dir+"/"+str(frame_count)+".jpg", face)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    except:
        print('error in detect_and_save_faces')

In [14]:
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    try:
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    except:
        output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

In [15]:
def draw_prediction(i,img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    global path
    print(y,y_plus_h,x, x_plus_w )
    output_dir1 = path+'final/person/'
    os.makedirs(output_dir1, exist_ok=True)
    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    crop_img = img[y:y_plus_h, x:x_plus_w]
    cv2.imwrite(output_dir1+str(i)+'.jpg',crop_img)
    
    try:
        resultImg,faceBoxes=highlightFace(faceNet,crop_img)
        if not faceBoxes:
            print("No face detected")
        else:
            
            for faceBox in faceBoxes:
                face=crop_img[max(0,faceBox[1]-padding):
                           min(faceBox[3]+padding,crop_img.shape[0]-1),max(0,faceBox[0]-padding)
                           :min(faceBox[2]+padding, crop_img.shape[1]-1)]

                blob=cv2.dnn.blobFromImage(face, 1.0, (227,227), MODEL_MEAN_VALUES, swapRB=False)
                genderNet.setInput(blob)
                genderPreds=genderNet.forward()
                gender=genderList[genderPreds[0].argmax()]
                print(f'Gender: {gender}')

                ageNet.setInput(blob)
                agePreds=ageNet.forward()
                age=ageList[agePreds[0].argmax()]
                print(f'Age: {age[1:-1]} years')

                cv2.putText(resultImg, f'{gender}, {age}', (faceBox[0], faceBox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2, cv2.LINE_AA)
                #cv2.imshow("Detecting age and gender", resultImg)
                f=open('E:/billboard/output/'+str(i)+'_gender.txt','w')
                f.write(str(gender))
                f.close()
                f=open('E:/billboard/output/'+str(i)+'_age.txt','w')
                if age=='(0-2)' or age== '(4-6)':
                    f.write('Child')
                elif age=='(8-12)' :
                    f.write('Teenager')
                elif age== '(15-20)':
                    f.write('Young Adult')
                elif age== '(25-32)' or age== '(38-43)':
                    f.write('Adult')
                else:
                    f.write('Senior')
                    
                        
                f.close()
                detect_and_save_faces(crop_img,i)
    except:
        print('Couldnt process person/'+str(i)+'.jpg')
    
    #detect_and_save_faces(crop_img,i)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [16]:
#path+'final/e.jpg'

In [17]:
image = cv2.imread(path+'final/e.jpg')

Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392

classes = None

In [18]:
path

'E:/billboard/'

In [19]:
with open(path+'object_opencv/yolov3.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

In [20]:
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))
colors=COLORS
net = cv2.dnn.readNet(path+'object_opencv/yolov3.weights', path+'object_opencv/yolov3.cfg')

blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

net.setInput(blob)

outs = net.forward(get_output_layers(net))

In [21]:
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4


In [22]:
for out in outs:
    for detection in out:
        scores = detection[0:2]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

In [23]:
def detect(image_file):
    global path
    try:
        image = cv2.imread(image_file)

        Width = image.shape[1]
        Height = image.shape[0]
        scale = 0.00392
        class_ids = []
        confidences = []
        boxes = []
        conf_threshold = 0.5
        nms_threshold = 0.4

        COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

        net = cv2.dnn.readNet(path+'object_opencv/yolov3.weights', path+'object_opencv/yolov3.cfg')

        blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

        net.setInput(blob)

        outs = net.forward(get_output_layers(net))
        #print(outs)
        for out in outs:

            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                #print('class_id',class_id)
                confidence = scores[class_id]
                if confidence > 0.5 and class_id==0:
                    center_x = int(detection[0] * Width)
                    center_y = int(detection[1] * Height)
                    w = int(detection[2] * Width)
                    h = int(detection[3] * Height)
                    x = center_x - w / 2
                    y = center_y - h / 2
                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    boxes.append([x, y, w, h])
        indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
        c=0
        for i in indices:
            try:
                box = boxes[i]
            except:
                i = i[0]
                box = boxes[i]

            x = box[0]
            y = box[1]
            w = box[2]
            h = box[3]
            #print('.....',class_ids[i])
            print(x,y,w,h)
            draw_prediction(c,image, class_ids[i], confidences[i], round(x-20), round(y-15), round(x+w+10), round(y+h+15))
            c=c+1
        #cv2.imshow("cam1", image)
        #cv2.waitKey()

        cv2.imwrite("object-detection1.jpg", image)
        cv2.imwrite("E:/BillBoard_Web/web/object-detection1.jpg", image)
        cv2.destroyAllWindows()
        results=[]
        for i in class_ids:
            print(classes[i])
            results.append(classes[i])
        return results,len(results)
    except:
        pass
            
        
    

In [24]:
#path+'final/e.jpg'

In [25]:

#detect(path+'final/10.jpg')

In [ ]:
cap = cv2.VideoCapture(1)
while True:
    ret, frame = cap.read()
    cv2.imwrite('input.jpg', frame)
    cv2.imwrite('E:/BillBoard_Web/web/live.png',frame)
    file_path='input.jpg'
    print('processing ',file_path)
    
    if not ret:
        break
    #cv2.imshow("Live Camera", frame)
    for file in os.listdir('E:/billboard/final/detected_faces/'):
        if os.path.isfile(os.path.join('E:/billboard/final/detected_faces/', file)):
            os.remove('E:/billboard/final/detected_faces/'+file)
    detect('E:/billboard/final/input.jpg')
    time.sleep(2)
    files = []
    for file in os.listdir('E:/billboard/final/detected_faces/'):
        if os.path.isfile(os.path.join('E:/billboard/final/detected_faces/', file)):
            files.append(file)
    c=0
    for f in files:
        print('f',f)
        face_recognition('E:/billboard/final/detected_faces/'+f,c)
        c=c+1
        time.sleep(2)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

processing  input.jpg
processing  input.jpg
252.5 -14.0 321 434
-29 435 232 584
processing  input.jpg
308.0 39.5 332 391
24 446 288 650
No face detected
person
person
person
person
person
processing  input.jpg
274.5 -1.0 357 464
-16 478 254 642
No face detected
9.0 20.5 334 415
6 450 -11 353
processing  input.jpg
140.0 100.0 268 370
85 485 120 418
No face detected
313.0 139.5 298 333
124 488 293 621
Gender: Female
Age: 25-32 years
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
processing  input.jpg
307.0 92.5 334 373
78 480 287 651
Gender: Male
Age: 25-32 years
95.0 70.5 258 389
56 474 75 363
No face detected
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
person
processing  input.jpg
84.0 29.0 298 424
14 468 64 392
No face detected
277.0 80.5 366 387
66 482 257 653
Gender: Male
Age: 25-32 years
316.0 49.0 204 312
34 376 296 530
No face detected
person
person
person

C:\Users\vinag\.conda\envs\myenv\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.


The Person is: 4
Current date is: 2024-04-23
Current Day: Tuesday
Current Time: 17:17:16
Week of the Month: 4
17 <class 'int'>
4 Teenager Male 2024-04-23 17-18 TUESDAY Week 4
processing  input.jpg
316.5 -3.5 273 351
-18 362 296 600
processing  input.jpg
357.5 8.0 281 452
-7 475 338 648
No face detected
person
person
person
person
processing  input.jpg
processing  input.jpg
processing  input.jpg
292.5 3.0 355 474
-12 492 272 658
No face detected
person
person
processing  input.jpg
294.5 -3.5 339 487
-18 498 274 644
No face detected
-0.5 169.0 239 302
154 486 -20 248
processing  input.jpg
-4.5 142.0 147 340
127 497 -24 152
processing  input.jpg
275.0 16.0 358 454
1 485 255 643
Gender: Male
Age: 25-32 years
131.5 377.5 169 101
362 494 112 310
No face detected
person
person
person
person
person
person
person
f 0.jpg
['0', '1', '2', '3', '4', '5', '6', '7']
E:/billboard/Dataset/0/0.jpg
E:/billboard/Dataset/1/0.jpg
E:/billboard/Dataset/2/0.jpg
E:/billboard/Dataset/3/0.jpg
E:/billboard/Datase

C:\Users\vinag\.conda\envs\myenv\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.


processing  input.jpg
360.0 146.5 212 327
132 488 340 582
Gender: Male
Age: 8-12 years
Gender: Female
Age: 8-12 years
133.0 154.0 254 314
139 483 113 397
Gender: Female
Age: 25-32 years
person
person
person
person
person
person
person
f 0.jpg
['0', '1', '2', '3', '4', '5', '6', '7']
E:/billboard/Dataset/0/0.jpg
E:/billboard/Dataset/1/0.jpg
E:/billboard/Dataset/2/0.jpg
E:/billboard/Dataset/3/0.jpg
E:/billboard/Dataset/4/0.jpg
E:/billboard/Dataset/5/0.jpg
E:/billboard/Dataset/6/0.jpg
E:/billboard/Dataset/7/0.jpg
done extraction
done training
inside for
The Person is: 3
Current date is: 2024-04-23
Current Day: Tuesday
Current Time: 17:18:20
Week of the Month: 4
17 <class 'int'>
3 Teenager Female 2024-04-23 17-18 TUESDAY Week 4


...

processing  input.jpg
385.5 71.5 257 409
56 496 366 652
No face detected
person
person
person
person
processing  input.jpg
318.0 35.0 328 424
20 474 298 656
No face detected
person
person
person
person
person
person
processing  input.jpg
384.5 236.0 251 238
221 489 364 646
No face detected
person
processing  input.jpg
254.5 -2.0 341 484
-17 497 234 606
No face detected
person
person
person
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
processing  input.jpg
383.0 58.5 248 419
44 492 363 641
No face detected
58.0 58.5 250 377
44 450 38 318
No face detected
p

C:\Users\vinag\.conda\envs\myenv\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.


processing  input.jpg
332.5 -6.0 253 352
-21 361 312 596
processing  input.jpg
280.0 130.5 252 317
116 462 260 542
Gender: Male
Age: 25-32 years
-9.5 59.0 247 342
44 416 -30 248
processing  input.jpg
378.5 1.0 259 472
-14 488 358 648
No face detected
142.5 -1.0 251 356
-16 370 122 404
processing  input.jpg
135.5 -2.5 253 357
-18 370 116 398
processing  input.jpg
318.0 16.0 320 432
1 463 298 648
No face detected
140.0 -3.0 246 358
-18 370 120 396
processing  input.jpg
321.0 18.5 310 427
4 460 301 641
No face detected
128.5 1.0 201 350
-14 366 108 340
processing  input.jpg
310.5 24.0 323 422
9 461 290 644
No face detected
135.5 -10.5 245 407
-26 412 116 390
processing  input.jpg
277.0 23.0 366 422
8 460 257 653
Gender: Male
Age: 25-32 years
116.5 14.5 249 357
0 386 96 376
Gender: Male
Age: 25-32 years
person
person
person
person
person
person
person
person
person
processing  input.jpg
169.5 47.0 239 288
32 350 150 418
Gender: Male
Age: 8-12 years
388.5 31.0 249 416
16 462 368 648
Gender:

C:\Users\vinag\.conda\envs\myenv\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.


processing  input.jpg
273.0 -2.5 194 271
-18 284 253 477
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279 247 479
processing  input.jpg
267.0 0.0 202 264
-15 279

In [ ]:













def download():
    url = 'http://192.168.0.140:4747/video'
    file_name = '1.png'

    res = requests.get(url, stream = True)

    if res.status_code == 200:
        with open(file_name,'wb') as f:
            shutil.copyfileobj(res.raw, f)
        print('Image sucessfully Downloaded: ',file_name)
    else:
        print('Image Couldn\'t be retrieved')

In [ ]:
download()

In [ ]:
detect(path+'final/1.png')


In [ ]:

while True:
    download()
    detect(path+'final/1.png')
    
    
    time.sleep(10)




In [ ]:
# Load camera
cap = cv2.VideoCapture('http://192.168.139.234:81/stream')

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    height, width, channels = frame.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    

    outs = net.forward(get_output_layers(net))
    count=0
    # Showing informations on the screen
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 :
                # Object detected
                count=count+1
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Draw rectangle and label
                cv2.rectangle(frame, (x, y), (x + w, y + h), colors[0], 2)
                cv2.putText(frame, classes[class_id], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[0], 2)
                if classes[class_id]!='':
                    print(classes[class_id])
                    f=open('output.txt','w')
                    f.write(str(classes[class_id]))
                    f.close()
                

    # Display the resulting frame
    #cv2.imshow('Camera '+str(count), frame)
    #print('Total count ',count)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
for f in os.walk('E:/2024/billboard/final/detected_faces/'):
    print(f)

In [ ]:
files = []
for file in os.listdir('E:/2024/billboard/final/detected_faces/'):
    if os.path.isfile(os.path.join('E:/2024/billboard/final/detected_faces/', file)):
        files.append(file)
for f in files:
    print('f',f)

In [ ]:
for file in os.listdir('E:/2024/billboard/final/detected_faces/'):
    if os.path.isfile(os.path.join('E:/2024/billboard/final/detected_faces/', file)):
        os.remove('E:/2024/billboard/final/detected_faces/'+file)